In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import json

# Define the path to your JSON file
file_path = '/content/drive/MyDrive/MasterThesis/captions_data_combined_all.json'

# Open and load the JSON data
with open(file_path, 'r') as file:
    captions_data = json.load(file)


In [ ]:
!pip install transformers torch evaluate

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline
import numpy as np

# Initialize the necessary components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Define the function to summarize captions
def summarize_captions(captions):
    combined_text = ' '.join(captions)
    summarized_text = summarizer(combined_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    return summarized_text

counter = 0
for key, item in captions_data.items():
    generated_captions = list(item['generated_captions'].values())
    summarized_caption = summarize_captions(generated_captions)

    # Split the summarized caption into sentences
    summarized_sentences = summarized_caption.split(". ")
    highest_similarity = -1
    most_similar_sentence = ""
    most_similar_caption_from_original = ""

    for sentence in summarized_sentences:
        inputs = clip_processor(text=[sentence] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity
        similarities = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1:]).detach().numpy()
        most_similar_index = similarities.argmax()
        if similarities[most_similar_index] > highest_similarity:
            highest_similarity = similarities[most_similar_index]
            most_similar_sentence = sentence
            most_similar_caption_from_original = item['original_coco_captions'][most_similar_index]

    # Update the JSON structure
    captions_data[key]['most_similar_summarized_sentence'] = most_similar_sentence
    captions_data[key]['most_similar_original_caption'] = most_similar_caption_from_original
    captions_data[key]['cosine_similarity_score'] = highest_similarity

    # Update counter and print
    counter += 1
    print(f"Item {counter}:")
    print(f"Most Similar Summarized Sentence: {most_similar_sentence}")
    print(f"Most Similar Original Caption: {most_similar_caption_from_original} (Score: {highest_similarity:.4f})\n")

# Save the modified JSON data
output_path = '/content/drive/MyDrive/MasterThesis/Scenario2_distilbart.json'
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


Your max_length is set to 130, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Item 1:
Most Similar Summarized Sentence:  A person in a black hoodie a hot dog on a bun with mustard and ketchup 
Most Similar Original Caption: some food is sticking out of a toy car (Score: 0.6094)



Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Item 2:
Most Similar Summarized Sentence:  a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic table 
Most Similar Original Caption: A picture of two hot dogs and a cup of soda (Score: 0.8254)



Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


...

Your max_length is set to 130, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


Item 490:
Most Similar Summarized Sentence: A man dressed as a chef in his chef uniform is seen in a TV costume .
Most Similar Original Caption: A person dressed up as a chef cooking. (Score: 0.8571)



Your max_length is set to 130, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


Item 491:
Most Similar Summarized Sentence: A man sat in front of the camera with a watch saying 'Iphone'
Most Similar Original Caption: a small bird with its head pulled back (Score: 0.6598)



KeyboardInterrupt: 

In [ ]:
!pip install transformers torch evaluate

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the necessary components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Define the function to summarize captions
def summarize_captions(captions):
    combined_text = ' '.join(captions)
    summarized_text = summarizer(combined_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    return summarized_text

# Define function to compute cosine similarities for each model vs original captions
def compute_similarities_for_each_model(generated_caption, original_captions):
    inputs = clip_processor(text=[generated_caption] + original_captions, return_tensors="pt", truncation=True, padding=True)
    embeddings = clip_model.get_text_features(**inputs)
    similarities = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1:]).detach().numpy()
    return similarities.tolist()

counter=0
for key, item in captions_data.items():
    generated_captions = item['generated_captions']
    original_captions = item['original_coco_captions']
    summarized_caption = summarize_captions(list(generated_captions.values()))

    # Calculate and store similarities for each model vs. original captions
    for model_name in ["blip", "gpt2", "pix2struct"]:
        model_caption = generated_captions[model_name]
        similarities = compute_similarities_for_each_model(model_caption, original_captions)
        item[f'{model_name}_vs_Orig_similarities'] = [float(sim) for sim in similarities]

    # Split the summarized caption into sentences and find the most similar sentence
    summarized_sentences = summarized_caption.split(". ")
    highest_similarity = -1
    most_similar_sentence = ""
    most_similar_caption_from_original = ""

    for sentence in summarized_sentences:
        inputs = clip_processor(text=[sentence] + original_captions, return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)
        similarities = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1:]).detach().numpy()
        most_similar_index = similarities.argmax()
        if similarities[most_similar_index] > highest_similarity:
            highest_similarity = similarities[most_similar_index]
            most_similar_sentence = sentence
            most_similar_caption_from_original = original_captions[most_similar_index]

    # Update the JSON structure with the most similar sentence info
    captions_data[key]['most_similar_summarized_sentence'] = most_similar_sentence
    captions_data[key]['most_similar_original_caption'] = most_similar_caption_from_original
    captions_data[key]['cosine_similarity_score'] =float(highest_similarity)

    print(most_similar_sentence)
    print(most_similar_caption_from_original)
    print(float(highest_similarity))
    print(counter)
    counter+=1

# Save the modified JSON data
output_path = '/content/drive/MyDrive/MasterThesis/Scenario2_distilbart_with_all_similarities.json'
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

...

 a woman preparing food a man in a kitchen cutting up food  A man is holding a bottle of wine and a bottle that says 'the  say 'the'
a man standing next to a kitchen counter preparing food.
0.7167207598686218
1086


Your max_length is set to 130, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


 A kitchen with a fridge and a fridge that says'sausages' A kitchen has a sink and a stove with a sink, sink, and refrigerator 
A kitchen with a lot of items sitting out. 
0.8195234537124634
1087


Your max_length is set to 130, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


is a kitchen that says 'V.V.' A man standing in a kitchen with his dog is a man standing with a dog 
A man points at the wall in his kitchen.
0.7499553561210632
1088
 A man is standing in front of a computer that says "Windows" a bathroom with a sink, mirror, and shower is a bathroom without a sink and mirror .
A bathroom with a sink, vanity and shower stall.
0.738386869430542
1089
JSON file has been updated and saved.


In [ ]:
!pip install transformers torch evaluate

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Function to summarize text
def summarize_text(text):
    summarized_text = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    return summarized_text

# New JSON data to store the similarity scores
similarity_scores_data = {}

counter=0
for key, item in captions_data.items():
    # Concatenate generated captions and summarize
    concatenated_generated_captions = " ".join(item['generated_captions'].values())
    summarized_generated_caption = summarize_text(concatenated_generated_captions)

    # Concatenate original captions, summarize
    concatenated_original_captions = " ".join(item['original_coco_captions'])
    summarized_original_caption = summarize_text(concatenated_original_captions)

    # Compute cosine similarity between summarized captions
    inputs = clip_processor(text=[summarized_generated_caption, summarized_original_caption], return_tensors="pt", padding=True, truncation=True)
    embeddings = clip_model.get_text_features(**inputs)
    similarity_score = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0)).item()

    # Store similarity score in new JSON data structure
    similarity_scores_data[key] = {
        "similarity_score": similarity_score,
        "summarized_generated_caption": summarized_generated_caption,
        "summarized_original_caption": summarized_original_caption
    }
    print(counter)
    counter+=1
    # Print the results
    print(f"Item {key}:")
    print(f"Summarized Generated Caption: {summarized_generated_caption}")
    print(f"Summarized Original Caption: {summarized_original_caption}")
    print(f"Similarity Score: {similarity_score}\n")
    print("="*50)

# Save the similarity scores to a new JSON file
output_path = '/content/drive/MyDrive/MasterThesis/similarity_scores_summary_vs_summary.json'
with open(output_path, 'w') as file:
    json.dump(similarity_scores_data, file, indent=4)

print("Similarity scores JSON file has been saved.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 941.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

...

1087
Item 1087:
Summarized Generated Caption:  A kitchen with a fridge and a fridge that says'sausages' A kitchen has a sink and a stove with a sink, sink, and refrigerator . A kitchen that says 'sausages', is a kitchen that has a fridge with a refrigerator that says it's 'ausages'.
Summarized Original Caption:  A cluttered kitchen with items strewn on the counter tops . A modern if messy modern kitchen white with red accents . A kitchen with a lot of items sitting out. a tiled kitchen with some appliances inside it .
Similarity Score: 0.8388991355895996



Your max_length is set to 130, but your input_length is only 72. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 130, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


1088
Item 1088:
Summarized Generated Caption:  A kitchen with a fridge that says V.V.T.B. is a kitchen that says 'V.V.' A man standing in a kitchen with his dog is a man standing with a dog . A man with a stove, a sink, and a refrigerator that says "V.B." is a refrigerator with a refrigerator .
Summarized Original Caption:  Two pans with cookies in them sitting on a counter with liquor bottles, a coffee maker and knives . Raw cookies in a pan on the counter and baked cookies on the stove .
Similarity Score: 0.5638807415962219



Your max_length is set to 130, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


1089
Item 1089:
Summarized Generated Caption:  A man is standing in front of a computer that says "Windows" a bathroom with a sink, mirror, and shower is a bathroom without a sink and mirror .
Summarized Original Caption:  A bathroom with a sink, vanity and shower stall is one of the most expensive bathrooms in the world . A bathroom vanity is a bathroom vanity with a small round sink . The bathroom has a shower stall, vanity, shower stall and bathroom vanity .
Similarity Score: 0.7614379525184631

Similarity scores JSON file has been saved.


In [3]:
!pip install transformers torch evaluate

import torch
import json
from transformers import CLIPProcessor, CLIPModel, pipeline

# Initialize the necessary components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Define the function to summarize captions
def summarize_captions(captions):
    combined_text = ' '.join(captions)
    input_length = len(combined_text.split())  # Calculate the input length

    # Dynamically adjust max_length with a gentler slope, ensuring it doesn't exceed 75% of the input length
    # but is also not shorter than a minimum viable length for coherence
    max_length_ratio = 0.75
    min_viable_length = 20  # Adjust this as needed to ensure summaries are not too short
    max_length = max(min_viable_length, int(input_length * max_length_ratio))

    # Ensure min_length is logical given the new max_length, aiming for summaries to have enough room to be coherent
    min_length = max(10, max_length // 2)

    summarized_text = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summarized_text

# Define function to compute cosine similarities for each model vs original captions
def compute_similarities_for_each_model(generated_caption, original_captions):
    inputs = clip_processor(text=[generated_caption] + original_captions, return_tensors="pt", truncation=True, padding=True)
    embeddings = clip_model.get_text_features(**inputs)
    similarities = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1:]).detach().numpy()
    return similarities.tolist()

counter=0
for key, item in captions_data.items():
    generated_captions = item['generated_captions']
    original_captions = item['original_coco_captions']
    summarized_caption = summarize_captions(list(generated_captions.values()))

    # Calculate and store similarities for each model vs. original captions
    for model_name in ["blip", "gpt2"]:
        model_caption = generated_captions[model_name]
        similarities = compute_similarities_for_each_model(model_caption, original_captions)
        item[f'{model_name}_vs_Orig_similarities'] = [float(sim) for sim in similarities]

    # Split the summarized caption into sentences and find the most similar sentence
    summarized_sentences = summarized_caption.split(". ")
    highest_similarity = -1
    most_similar_sentence = ""
    most_similar_caption_from_original = ""

    for sentence in summarized_sentences:
        inputs = clip_processor(text=[sentence] + original_captions, return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)
        similarities = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1:]).detach().numpy()
        most_similar_index = similarities.argmax()
        if similarities[most_similar_index] > highest_similarity:
            highest_similarity = similarities[most_similar_index]
            most_similar_sentence = sentence
            most_similar_caption_from_original = original_captions[most_similar_index]

    # Update the JSON structure with the most similar sentence info
    captions_data[key]['most_similar_summarized_sentence'] = most_similar_sentence
    captions_data[key]['most_similar_original_caption'] = most_similar_caption_from_original
    captions_data[key]['cosine_similarity_score'] =float(highest_similarity)

    print(most_similar_sentence)
    print(most_similar_caption_from_original)
    print(float(highest_similarity))
    print(counter)
    counter+=1

# Save the modified JSON data
output_path = '/content/drive/MyDrive/MasterThesis/Scenario2_distilbart_with_all_similarities_only_2.json'
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 432.7 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

 a person in a black hoodie a hot dog on a bun with mustard and k
some food is sticking out of a toy car
0.6095834374427795
0
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic
A picture of two hot dogs and a cup of soda
0.859606146812439
1
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic
A picture of two hot dogs and a cup of soda
0.859606146812439
2
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic
A picture of two hot dogs and a cup of soda
0.859606146812439
3
 a table with a plate of hot dogs and a cup of soda a hot dog and a drink on a picnic
A picture of two hot dogs and a cup of soda
0.859606146812439
4
 a group of people standing around a motorcycle a person riding a motorcycle with a dog on the back 
A group of people watch a dog ride a motorcycle. 
0.9207561612129211
5
 A group of people riding scoots on a street a number of people on a motorcycle 
A group of people rid